# <div align="center"> Loss Functions </div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.18.5
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [2]:
###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')


## NLLLoss vs CrossEntropyLoss

In [3]:
X = torch.randn(2, 4)
y = torch.arange(0, 2)
X, y, X.shape, y.shape

(tensor([[-1.0619, -0.1077, -0.6097, -1.1655],
         [ 1.5475, -0.6481,  1.1606, -0.7103]]),
 tensor([0, 1]),
 torch.Size([2, 4]),
 torch.Size([2]))

### softmax range:(0, 1)

In [4]:
soft_output = F.softmax(X, dim=1)  # nn.Softmax(dim=1)
soft_output, soft_output.shape

(tensor([[0.1647, 0.4278, 0.2589, 0.1485],
         [0.5277, 0.0587, 0.3584, 0.0552]]),
 torch.Size([2, 4]))

### log_softmax range:(-inf, 0)

In [5]:
log_output = F.log_softmax(X, dim=1)  # nn.LogSoftmax(dim=1)
log_output, log_output.shape, torch.log(soft_output)

(tensor([[-1.8034, -0.8491, -1.3511, -1.9070],
         [-0.6393, -2.8349, -1.0261, -2.8970]]),
 torch.Size([2, 4]),
 tensor([[-1.8034, -0.8491, -1.3511, -1.9070],
         [-0.6393, -2.8349, -1.0261, -2.8970]]))

### nllloss

In [6]:
nlloss_output = F.nll_loss(log_output, y)  # nn.NLLLoss()
nlloss_output

tensor(2.3191)

### cross_entropy = log_softmax +  nllloss

In [7]:
ce_output = F.cross_entropy(X, y)
ce_output

tensor(2.3191)

In [8]:
X = torch.randn(4, 64, 32)
y = torch.LongTensor(torch.randint(low=0, high=32, size=(4, 64)))
X.shape, y.shape

(torch.Size([4, 64, 32]), torch.Size([4, 64]))

In [9]:
ce_output = F.cross_entropy(X.transpose(1, 2), y)
ce_output

tensor(3.8459)

In [29]:
X_T = X.transpose(1, 2)

torch.argmax(X_T, dim=1).shape

torch.Size([4, 64])

In [26]:
y = -100
for i in range(32):
    x = X_T[0][i][1]
    print(i, x)

0 tensor(-0.9854)
1 tensor(0.2662)
2 tensor(0.1201)
3 tensor(1.1647)
4 tensor(0.3448)
5 tensor(-0.4662)
6 tensor(0.3597)
7 tensor(-0.2521)
8 tensor(-0.2011)
9 tensor(-0.3289)
10 tensor(0.2396)
11 tensor(0.2461)
12 tensor(0.7395)
13 tensor(-1.0237)
14 tensor(0.4662)
15 tensor(0.0393)
16 tensor(0.4095)
17 tensor(-0.0378)
18 tensor(0.3668)
19 tensor(-0.0016)
20 tensor(-0.4012)
21 tensor(-2.2986)
22 tensor(0.9346)
23 tensor(-0.0021)
24 tensor(0.1083)
25 tensor(0.1763)
26 tensor(1.1837)
27 tensor(-0.4458)
28 tensor(-1.7021)
29 tensor(-0.7174)
30 tensor(0.9616)
31 tensor(-0.5874)


In [20]:
y

tensor(1.5204)

```
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(*list(models.resnet50(pretrained=True).children())[:-3])
        self.temporal_context = nn.Sequential(nn.Conv3d(in_channels=1024,
                                out_channels=512,kernel_size=3,padding=[1,0,0]), nn.ReLU())
   

    def forward(self,x):
        b, f, c, h, w = x.size()
        features = self.cnn(x.view(b*f, c, h, w))
        temp_features = self.temporal_context(features.unsqueeze(0).view(b,features.shape[1], f,features.shape[2],features.shape[3]))
        out = nn.AvgPool2d(5)(temp_features.view(b*f,512, temp_features.shape[-1], temp_features.shape[-1])).view(b, f, -1)
        return out

class PeriodPredictor(nn.Module):

    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3), nn.ReLU())
        self.projection = nn.Linear(1922,512)
        self.transformer = nn.Transformer(nhead=4,dim_feedforward=128)

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(x.size(0), 64, -1)
        out = self.projection(out)
        out = self.transformer(out, out)
        return out

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.l_network = nn.Sequential(nn.Linear(512,512),nn.ReLU(),nn.Linear(512,32))
        self.p_network = nn.Sequential(nn.Linear(512,512),nn.ReLU(),nn.Linear(512,1))

    def forward(self, x):
        b, f, e = x.size()
        l = self.l_network(x.view(b*f,-1))
        p = self.p_network(x.view(b*f,-1))
        return l.view(b,-1), p.view(b,-1)

class RepNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.period_predictor = PeriodPredictor()
        self.classifier = Classifier()
        self.optimizer = torch.optim.SGD(params=self.parameters(),  lr=1e-3, momentum=0.9, weight_decay=5e-4)

    def forward(self, x):
        features = self.encoder(x)
        sim_mat = -torch.matrix_power(torch.cdist(features, features), 2).unsqueeze(1)
        period = self.period_predictor(sim_mat)
        l, p = self.classifier(period)
        return l, p

    def train_epoch(self, loader, epoch, device):
        self.train()
        print(
            "\n" + " " * 10 + "****** Epoch {epoch} ******\n"
            .format(epoch=epoch)
        )

        training_losses = []
        mae = deque(maxlen=30)
        self.optimizer.zero_grad()
        with tqdm(total=len(loader), ncols=80) as pb:


            for batch_idx, d in enumerate(loader):
                frames, l, p = d
                frames, l, p = frames.to(device), l.to(device), p.to(device)
                frames.requires_grad = True
                self.optimizer.zero_grad()
                l_logits, p_logits = self.forward(frames)
                loss = torch.nn.BCELoss()(torch.nn.Sigmoid()(p_logits.view(-1)),p.float().view(-1))
                loss += torch.nn.CrossEntropyLoss()(l_logits.view(frames.shape[0]*64,32), l.view(-1))
                training_losses.append(loss.data.cpu().numpy())
                loss.backward()
                clip_gradient(self.optimizer, 0.1)
                self.optimizer.step()
                counts_t = []
                counts_p = []
                p_preds = torch.where(torch.nn.Sigmoid()(p_logits) > 0.5, torch.tensor(1).cuda(), torch.tensor(0).cuda())
                l_preds = l_logits.view(frames.shape[0],64,32).argmax(2)
                for l_i, p_i, ll_i, pl_i in zip(l, p, l_preds, p_preds):
                    reps = torch.where(l_i>0)[0]
                    counts_t.append( torch.sum(torch.div(p_i[reps].float(),l_i[reps].float())).data.cpu().numpy())
                    counts_p.append( torch.sum(torch.div(pl_i[reps].float(),ll_i[reps].float())).data.cpu().numpy())
                try:
                    mae_i = mean_absolute_error(np.array(counts_t), np.array(counts_p))
                    mae.append(mae_i)
                    pb.update()
                    pb.set_description(
                        f"Loss: {loss.item()}, MAE: {np.mean(mae)}")

                except Exception as e:
                    print(e)
                    continue


    def validate(self, loader):
        self.eval()
        with tqdm(total=len(loader), ncols=80) as pb:
            for batch_idx, frames, l, p in enumerate(loader):
                l_logits, p_logits = self.forward(frames)
                loss = torch.nn.CrossEntropyLoss()(l_logits, l) + torch.nn.CrossEntropyLoss()(p_logits, p)
                pb.update()
                pb.set_description(
                    "Loss: {:.4f}".format(
                        loss.item()))```
```